In [3]:
# File imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import csv
from collections import defaultdict
import codecs
import re
from datetime import datetime
#from search_term import give_med_terms

In [5]:
number = r'\d{2,3}'
gender = r'(\b[Mm]ale?)|(\b[Ff]emale?)|(\bFEMALE)|(\bMALE)|(/b)|F/|M/'
Date = r'(([A-Z0-9][A-Z0-9]?[/-])?[A-Z0-9][A-Z0-9]?[/-][A-Z0-9][A-Z0-9][A-Z0-9]?[A-Z0-9]?)|([A-Za-z][A-Za-z][A-Za-z]\s..?[,]\s....)'
DOB  = r'(.*)?DOB|[Dd][aA][tT][eE]\s[oO][fF]\s[Bb][iI][rR][tT][hH]\s?(.*)?'
year_four_digit = r'\b(19|20)\d{2}(w+)?'
year_two_digit = r'\d{2}$(w+)?'
product_type = r'(\b[Pp]roduct\s[Tt]ype):\s?.*'
permanent = r'[Pp][eE][rR][mM]([aA][nN][aA][nN][tT])?'
term = r'[tT][eE][rR][mM]'

#Assuming USA currency dollar
amount_with_dollar = r'(\$\s?\d{1,3}(,\d{2,3})*(\.\d+)?)(\s?[kK]?)(\s?[mM]?[mM]?(illion)?(ILLION)?)([bB]?)'
amount_without_dollar = r'(\$?\s?\d{1,3}(,\d{2,3})*(\.\d+)?)(\s?[kK]?)(\s[mM]?[mM]?(illion)?(ILLION)?)([bB]?)((\s?[Yy][Ee][aA][rR][sS]?)?)'
faceamount = r'(\b[Ff]ace\s?[Aa]mount:?\s?.*)'
termamount = r'(.*)?[Tt][eE][rR][mM](.*)?'   			#Regex to read single line from first newline to next newline
seeking = r'(.*)?[Ss][eE][eE][kK]([iI][nN][gG])?(.*)?'
term_year = r'(y(ea)?r|Y(ea)?r|Y(ea)?r)'
k_conv = r'(\s?[kK])'
m_conv = r'(\s?[mM][mM]?(illion)?(ILLION)?)'
num_conv = r'\d{1,3}'

weight = r'(.*)?\b[wW][eE][iI][gG][hH][tT]\s?(.*)?' 
weight_num = r'(\d*\.?\d+)\s?(lb|lbs|Lbs|LB|LBS|kg|Kg|KG|#)'		#r'(.*)\s?([lL][bB][sS]|[oO][zZ]|[gG]|[kK][Gg])' 

age_simple = r'(.*)?[Aa][Gg][Ee]\s?(.*)?'
age = r'(.*\s?[Yy]([eE][aA])?[rR]?[sS]?\s?([oO][lL][dD])?)'
age_from_gender = r'(.*)?(\b[Mm]ale?)|(\b[Ff]emale?)|(\bFEMALE)|(\bMALE)|(/b)\s?(.*)?' 

height_num = r'\d{1,2}'
height1 = r'((.*)?\s?([Ff][eE][eE][tT])((.*)?\s?([iI][nN][Cc][Hh][Ee][Ss]))?)'
height2 = r'.[\'|\’](\s?.[\"|\”])?' 
feet = r'\d[\'|\’]'
inches = r'\d[\"|\”]' 

preferred = r'(.*)?(Preferred|preferred)\s?(.*)?'
height_word = r'Height|height'
weight_word = r'Weight|weight' 

build = r'(Build|build)\s?(.*)?'
build_weight = r'\d{3}'
build_height = r'\d\.\d'

smoker = r'(.*)?[sS][Mm][oO][Kk]\s?(.*)?' 
tobacco = r'(.*)?[Tt][oO][bB][aA][cC][cC][oO]\s?(.*)?'
no = r'[nN][oO]'

med = r'(.*)?\b[mM][eE][dD][iI][cC][aA][tT][iI][oO][nN]\s?(.*)?'

family = r'(.*)?(\b[Ff]amily)\s?(.*)?'
family_member = r'(.*)?(\b[Mm]om)|(\b[Ff]ather)|(\b[Dd]ad)|(\b[Ss]ister)|(\b[Bb]rother)|(\b[Hh]usband)|(\b[Ww]ife)\s?(.*)?'

lives = r'(.*)?(\b[Ll]ives)\s?(.*)?'
prop = r'(.*)?(\b[Pp]roperty)\s?(.*)?'

In [6]:
def reg(st,i):
    for line in st: #iterate through every line
        #return list of entities in that line
        num = re.search(number, line, re.I | re.U)
        x=0
        x = re.search(Date, line, re.I | re.U)
#Gender
        y = re.search(gender, line, re.I | re.U)
        if(y):
            if(y.group(0)=='F/' or y.group(0)=='f/'):
                data[i][0]='Female'
            elif(y.group(0)=='M/' or y.group(0)=='m/'):
                data[i][0]='Male'
            else:

                data[i][0]=(y.group(0))
        elif(y and num):
            data[i][0]=(y.group(0))
        else:
            data[i][0]=" "

#Year for DOB
        z = 0
        x1 = re.search(year_four_digit, line, re.I | re.U)
        if(x):
            x1 = re.search(year_four_digit, x.group(0), re.I | re.U)
            x2 = re.search(year_two_digit, x.group(0), re.I | re.U)
            if(x1):
                data[i][1]=x1.group(0)
            elif(x2):
                z = x2.group(0)
                data[i][1] = '19'+z
        elif(x1):
            x1 = re.search(year_four_digit, line, re.I | re.U)

            data[i][1]=x1.group(0)
        else:
            data[i][1]=" "

#Age in years
        age_reg = re.search(age, line, re.I | re.U)
        age_simple_reg = re.search(age_simple, line, re.I | re.U)
        dob = re.search(DOB, line, re.I | re.U)
        age_gender_reg = re.search(age_from_gender, line, re.I | re.U)

        if(age_gender_reg):
                am = re.search(number, age_gender_reg.group(0), re.I | re.U)
                if(am):
                    data[i][2]=am.group(0)

        if(x1):#20/03/1996

            currentYear = datetime.now().year

            data[i][2]=((currentYear-(int)(x1.group(0))))

        else:
            if(x1 and dob):#DOB 20/03/1996

                currentYear = datetime.now().year

                data[i][2]=((currentYear-(int)(x1.group(0))))
            elif(x1 and y):#Male 20/03/1996

                currentYear = datetime.now().year

                data[i][2]=((currentYear-(int)(x1.group(0))))

            else:
                data[i][2]=' '

            if(age_reg):#20 years ago
                age_num = age_reg.group(0)
                an = re.search(number, age_num, re.I | re.U)
                if(an):

                    data[i][2]=(an.group(0))

            if(age_simple_reg):#Age 20
                age_num = age_simple_reg.group(0)
                an = re.search(number, age_num, re.I | re.U)
                if(an):

                    data[i][2]=(an.group(0))

            if((data[i][2]<'18') and data[i][1]!=" "):#From Year of Birth
                currentYear = datetime.now().year
                data[i][2]=(currentYear-(int)(data[i][1]))

#Product Type
        z=re.search(product_type, line, re.I | re.U)
        perm_reg = re.search(permanent, line, re.I | re.U)
        term_type_reg = re.search(term, line, re.I | re.U)
        if(z): 

            data[i][3]=(z.group(0))
        elif(perm_reg):
            final_str = "Product Type: Permanent"
            data[i][3]=(final_str)
        elif(term_type_reg):
            final_str = "Product Type: Term"
            data[i][3]=(final_str)
        else:
            data[i][3]=" "
            
        w = re.search(faceamount, line, re.I | re.U)
        term_reg = re.search(termamount, line, re.I | re.U)
        seek_reg = re.search(seeking, line, re.I | re.U)
#With faceAmount
        if(w):
            k = re.search(k_conv, w.group(0), re.I | re.U)
            if(k):
                nn = re.search(num_conv, w.group(0), re.I | re.U)
                if(nn):
                    data[i][4]='Face Amount: $'+((nn.group(0))+',000')
            else:
                data[i][4]=(w.group(0))


#With term Amount
        elif(term_reg):
            amd = re.search(amount_with_dollar, term_reg.group(0), re.I | re.U)
            amwd = re.search(amount_without_dollar, term_reg.group(0), re.I | re.U)			#Find 2nd regex in the same line of 1st regex 
            z = 0
            if(amd):
                k = re.search(k_conv, amd.group(0), re.I | re.U)
                m = re.search(m_conv, amd.group(0), re.I | re.U)
                if(k):
                    nn = re.search(num_conv, amd.group(0), re.I | re.U)
                    data[i][4]='Face Amount: '+((nn.group(0))+',000')
                elif(m):
                    nn = re.search(num_conv, amd.group(0), re.I | re.U)
                    data[i][4]='Face Amount: '+((nn.group(0))+',000,000')
                else:
                    data[i][4]='Face Amount: '+amd.group(0)
            elif(amwd):
                term_year_reg = re.search(term_year, amwd.group(0), re.I | re.U)
                if(term_year_reg):
                    data[i][4]='Term Year: '+(amwd.group(0))
                else:
#data[i][4]='Face Amount: $'+(amwd.group(0))
                    k = re.search(k_conv, amwd.group(0), re.I | re.U)
                    m = re.search(m_conv, amwd.group(0), re.I | re.U)
                    if(k):
                        nn = re.search(num_conv, amwd.group(0), re.I | re.U)
                        data[i][4]='Face Amount: $'+((nn.group(0))+',000')
                    elif(m):
                        nn = re.search(num_conv, amwd.group(0), re.I | re.U)
                        data[i][4]='Face Amount: $'+((nn.group(0))+',000,000')
                    else:
                        data[i][4]='Face Amount: $'+amwd.group(0)
#With Seeking
        elif(seek_reg):
            amd = re.search(amount_with_dollar, seek_reg.group(0), re.I | re.U)
            amwd = re.search(amount_without_dollar, seek_reg.group(0), re.I | re.U)			#Find 2nd regex in the same line of 1st regex 
            if(amd):
#data[i][4]='Face Amount: '+(amd.group(0))
                k = re.search(k_conv, amd.group(0), re.I | re.U)
                m = re.search(m_conv, amd.group(0), re.I | re.U)
                if(k):
                    nn = re.search(num_conv, amd.group(0), re.I | re.U)
                    data[i][4]='Face Amount: '+((nn.group(0))+',000')
                elif(m):
                    nn = re.search(num_conv, amd.group(0), re.I | re.U)
                    data[i][4]='Face Amount: '+((nn.group(0))+',000,000')
                else:
                    data[i][4]='Face Amount: '+amd.group(0)
            elif(amwd):
                term_year_reg = re.search(term_year, amwd.group(0), re.I | re.U)
                if(term_year_reg):
                    data[i][4]='Term Year: '+(amwd.group(0))
                else:
#data[i][4]='Face Amount: $'+(amwd.group(0))
                    k = re.search(k_conv, amwd.group(0), re.I | re.U)
                    m = re.search(m_conv, amwd.group(0), re.I | re.U)
                    if(k):
                        nn = re.search(num_conv, amwd.group(0), re.I | re.U)
                        data[i][4]='Face Amount: $'+((nn.group(0))+',000')
                    elif(m):
                        nn = re.search(num_conv, amwd.group(0), re.I | re.U)
                        data[i][4]='Face Amount: $'+((nn.group(0))+',000,000')
                    else:
                        data[i][4]='Face Amount: $'+amwd.group(0)
        else:
            data[i][4]=" "


#Weight
        x=re.search(weight_num, line, re.I | re.U) 
        wt=re.search(weight, line, re.I | re.U)
        if(x): 
#print (x.group(0)+"\n")
            data[i][5]=(x.group(0))
        elif(wt):
            am = re.search(weight_num,wt.group(0), re.I | re.U)
            if(am):
                data[i][5]=(am.group(0))
        else:
            data[i][5]=" "

#Height
        ht = re.search(height1, line, re.I | re.U)
        htsym = re.search(height2, line, re.I | re.U)
        if(ht): 

            data[i][6]=(ht.group(0))
        elif(htsym):
            f = re.search(feet, (htsym.group(0)), re.I | re.U)
            inch = re.search(inches, (htsym.group(0)), re.I | re.U)
            if(f):

                am = re.search(height_num, (f.group(0)), re.I | re.U).group(0) + ' Feet'
                if(i):
                    am+=re.search(height_num, (inch.group(0)), re.I | re.U).group(0) + ' Inches' 
                data[i][6]=am
        else:
            data[i][6]=" "

#Preferred Height & Weight
        pr = ''
        pr = re.search(preferred, line, re.I | re.U)
        if(pr!='' and pr):
            h_reg = re.search(height_word, pr.group(0), re.I | re.U)
            if(h_reg):
                data[i][6] = "5 Feet 9 Inches"
            w_reg = re.search(weight_word, pr.group(0), re.I | re.U)
            if(w_reg):
                data[i][5] = "196 lbs"

#Height & Weight from build
        bu = ''
        bu = re.search(build, line, re.I | re.U)
        if(bu):
            h_reg = re.search(build_height, bu.group(0), re.I | re.U)
            if(h_reg):
                data[i][6] = h_reg.group(0) + ' Feet'
                h =' '
            w_reg = re.search(build_weight, bu.group(0), re.I | re.U)
            if(w_reg):
                data[i][5] = w_reg.group(0)+ ' lbs'
                w = ' '

#Habit
        sm = re.search(smoker, line, re.I | re.U)
        tob = re.search(tobacco, line, re.I | re.U)
        if(sm): 
            if(re.search(no, sm.group(0), re.I | re.U)):
                data[i][7]="Non-Smoker"
            else:
                data[i][7]="Smoker"
        elif(tob):

            if(re.search(no, tob.group(0), re.I | re.U)):
                data[i][7]="Non-Tobacco"
            else:
                data[i][7]="Tobacco"
        else:
            data[i][7]=" "


#Medication & Treatment
        med_reg = (re.search(med,line, re.I | re.U))
        if(med_reg):
            if(re.search(no, med_reg.group(0), re.I | re.U)):
                data[i][8]="No Medication"
                
        else:#Write else outsite condition (to stop rewriting of above cell)
            data[i][8]=""

#Family
        family_reg = (re.search(family,line, re.I | re.U))
        family_member_reg = (re.search(family_member,line, re.I | re.U))
        if(family_reg):
            data[i][9]=family_reg.groups()
            
        else:#Write else outsite condition (to stop rewriting of above cell)
            data[i][9]=""

#Property
        lives_reg = (re.search(lives,line, re.I | re.U))
        prop_reg = (re.search(prop,line, re.I | re.U))
        if(lives_reg):
            data[i][10]=lives_reg.groups()
            if(prop_reg):
                data[i][10]=lives_reg.groups()+prop_reg.groups()
        elif(prop_reg):
            data[i][10]=prop_reg.groups()
            
        else:#Write else outsite condition (to stop rewriting of above cell)
            data[i][10]=""

    data[i][11]=st
#medical data
        #data[i][11] = give_med_terms(line)

    wtr.writerows(data)




i=0
w, h = 12, 1;
data = [[" " for x in range(w)] for y in range(h)]
st = []

out = open('regexProcessed.csv', 'w', newline='')
wtr= csv.writer( out )
wtr.writerow(['Gender','Year_of_birth','Age(years)','Product Type','Face Amount','Weight','Height','Habit','Medication','Family','Property',''])



with open('raw_data1.csv','r',encoding="ISO-8859-1") as f:
    rows = csv.reader(f)
    for row in rows:

        st.append(row[8])
        reg(st,i)
        st=[]
          

out.close()

In [7]:
df = pd.read_csv("regexProcessed.csv", encoding='ISO-8859-1')
df = df.iloc[1:]
df

,Gender,Year_of_birth,Age(years),Product Type,Face Amount,Weight,Height,Habit,Medication,Family,Property,Unnamed: 11
1,FEMALE,1977,41,Product Type: Permanent,"Face Amount: 50,000",200KG,,,NaN,NaN,NaN,"['Gender: FEMALE\nDOB : APR 20, 1977\nProduct ..."
2,male,1921,97,,"Face Amount: 250,000",,,,NaN,NaN,NaN,['Gender: male\nDOB : 10/00/1921\nFace Amount:...
3,Male,1997,21,,,200 lb,5 Feet,,No Medication,"('No significant ', 'family', 'history')",NaN,"[""Male\n01/31/97\n5' 6Ã\n200 lb\nNo medicatio..."
4,Male,1945,73,,"Face Amount: $20,000",199#,8 Feet,Non-Tobacco,NaN,"('', 'Family', 'hx of RA in Mother and Sister....",NaN,"[""Seeking $20,000 LN offered Table C due to ch..."
5,Male,,,Product Type: Permanent,,,,,NaN,NaN,NaN,['Male 53\nBuilt Ã¢\x80\x93 Preferred NT\nDiag...
6,Female,1966,52,Product Type: Term,"Face Amount: $500,212",,,Tobacco,NaN,"('Strong ', 'family', 'history for skin cancer...",NaN,['General:\n - Client Gender: Female\n ...
7,Female,1996,22,Product Type: Term,"Face Amount: 50,000,000",,,Non-Tobacco,NaN,NaN,"('', 'Lives', 'in XXXX XXXX, She is a US Citiz...","['Female, DOB 01/03/1996NS looking for $50m te..."
8,MALE,1950,68,,"Face Amount: $300,000",,,,NaN,NaN,NaN,['Subject: [External] QQ: S BarXXXX\n\nPlease ...
9,Female,1950,68,,,255lb,,,NaN,NaN,NaN,['Subject: [External] Pascarella Case - Potent...
10,Female,1950,35,Product Type: Term,"Face Amount: $777,000",199 lbs,6.6 Feet,,NaN,NaN,"('Household income is 11K, she is a home maker...","['Subject: [External] XXXXX\n\nF/age 35, NS, b..."


In [8]:
def changeval(ans):
    
    val = re.sub("Face Amount: ",' ',ans)
    
    val = re.sub(",",' ',val)
    val = str(re.sub(" ",'',val))
    index = (val.find('$'))
    if(index!=-1):
        val = val[1:]
        
    #val = int(val)
    return (val)


df['Normalized_Face'] = df['Face Amount'].apply(changeval)
df['Normalized_Face']

1                50000
2               250000
3                     
4                20000
5                     
6               500212
7             50000000
8               300000
9                     
10              777000
11              555000
12    TermYear:22years
13                    
14            59000000
15             1555000
16                    
17            20000000
18     TermYear:20year
19                    
Name: Normalized_Face, dtype: object

In [9]:
df['ColumnA'] = df[df.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
df['ColumnA']

1     FEMALE,1977,41,Product Type: Permanent,Face Am...
2     male,1921,97, ,Face Amount: 250,000, , , ,['Ge...
3     Male,1997,21, , ,200 lb,5 Feet, ,No Medication...
4     Male,1945,73, ,Face Amount: $20,000 ,199#,8 Fe...
5     Male, , ,Product Type: Permanent, , , , ,['Mal...
6     Female,1966,52,Product Type: Term,Face Amount:...
7     Female,1996,22,Product Type: Term,Face Amount:...
8     MALE,1950,68, ,Face Amount: $300,000, , , ,['S...
9     Female,1950,68, , ,255lb, , ,['Subject: [Exter...
10    Female,1950,35,Product Type: Term,Face Amount:...
11    Male, , ,Product Type: Term,Face Amount: $555,...
12    Male, ,22,Product Type: Term,Term Year:  22 ye...
13    Female, ,33, , , , ,Non-Tobacco,['Subject: [Ex...
14     ,1995,23,Product Type: Term,Face Amount: $59,...
15    Male,1995,55,Product Type: Term,Face Amount: $...
16     ,1995,23, , , , , ,['APS SUMMARY\nPMH: Subara...
17    Female,1996,22,Product Type: Term,Face Amount:...
18    Female,1996,55,Product Type: Term,Term Yea

In [10]:
# Reading the Pre-Processed File



#process_data(name)

#df = pd.read_csv("Processed_Data.csv", encoding='ISO-8859-1')
#df["Processed"][0]


In [11]:
from nltk.corpus import stopwords,wordnet as wn
from nltk.tokenize import wordpunct_tokenize,sent_tokenize
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import re


In [12]:
#Removes all punctuations which acts as noise

def rem_punt(doc):
    ans = re.sub('"|\\n|\(|\)|\.|[$!--+@#:]',' ',doc)
    ans = re.sub(' +',' ',ans)
    ans = ans.lower()
    return ans


# Stop words removal using tokenization

stop_word = set(stopwords.words('english'))

def tokenize(document): 
    lemmy = []
    for sent in sent_tokenize(document):
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            #print(token,tag)
            if token in stop_word:
                 continue
            lemma = lemmatize(token, tag)
            lemmy.append(lemma)
    return lemmy

#Lemmatization for tokens simplification

def lemmatize(token, tag):
    tag = {
          'N': wn.NOUN,
          'V': wn.VERB,
          'R': wn.ADV,
          'J': wn.ADJ
    }.get(tag[0], wn.NOUN)
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(token, tag)

In [13]:
# In[22]:

df['Lemmitize'] = df['ColumnA'].apply(rem_punt).apply(tokenize)


df.to_csv('NLPProcessed.csv',index=False, encoding = "utf-8")

In [14]:
# In[25]:

df = pd.read_csv('NLPProcessed.csv')


# # Statistical Modeling 

# In[26]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer,LabelEncoder
from sklearn.metrics import accuracy_score,classification_report


# In[28]:

X = df['Lemmitize']
of = pd.read_csv('raw_data1.csv', encoding='ISO-8859-1')
y = of['Offer']
#y = df['Offer_noise_free']
#lab_y = LabelEncoder()
#y = lab_y.fit_transform(y)

In [15]:
X[0]

"['female', '1977', '41', 'product', 'type', 'permanent', 'face', 'amount', '50', '000', '200kg', '[', 'gender', 'female', '\\\\', 'ndob', 'apr', '20', '1977', '\\\\', 'nproduct', 'type', 'permanent', '\\\\', 'nface', 'amount', '50', '000', '\\\\', 'n200kg', '\\\\', 'n', '\\\\', 'nhad', 'bypass', '2010', '\\\\', 'ntakes', 'follow', 'medication', '\\\\', 'n1', 'metoprolol', 'blood', 'pressure', 'diagnose', '2014', 'take', '25', 'mg', 'daily', '\\\\', 'n2', 'levothyroxine', 'thyroid', 'diagnose', '2016', 'take', '25', 'mg', 'daily', '\\\\', 'n', ']', '50000']"

In [16]:
# In[29]:

X_train,X_test,y_train,y_test = train_test_split(X,y)


# In[30]:

vect = TfidfVectorizer(max_df=0.8, max_features=15000, min_df=0.01, use_idf=True , ngram_range=(1,3))

In [17]:

#from xgboost.sklearn import XGBClassifier
#model1 = XGBClassifier(nthread=4,n_estimators=1000)


# Naive Bayes

from sklearn.naive_bayes import GaussianNB,MultinomialNB
model2 = GaussianNB()


# ExtraTree Classifier

from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
model3 = RandomForestClassifier(n_estimators=600,n_jobs=-1)

# SVM Classifier

from sklearn.svm import SVC
model4 = SVC()


# Logistic Regression 

from sklearn.linear_model import LinearRegression,SGDClassifier,LogisticRegression
model5 = LogisticRegression()

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model6 = LinearDiscriminantAnalysis()
'''
import matplotlib.pyplot as plt 
%matplotlib inline
plt.spy(vect)
'''

'\nimport matplotlib.pyplot as plt \n%matplotlib inline\nplt.spy(vect)\n'

In [18]:
# Model Fitting

import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import time

name = [] 
results = []
matrix_confusion = []
training_time = []
prediction_time = []
def model_making(model_name, vect , model , X_train , y_train , X_test , y_test):
    
    t1 =time.time()
    clf = make_pipeline(vect,model)
    clf.fit(X_train,y_train)
    t2 = time.time()
    training_time.append(t2-t1)
    
    t1 = time.time()
    pd = clf.predict(X_test)
    t2 = time.time()
    prediction_time.append(t2-t1)
    
    y_pred = clf.predict(X_test)
    name.append(model_name)
    results.append(accuracy_score(y_test, y_pred)*100)
    matrix_confusion.append(confusion_matrix(y_test, y_pred))
    
    #print ("=====Accuracy Score ", "{0:.2f}".format(accuracy_score(y_test, y_pred)*100), "%")
    #print ("=====Confusion Matrix")
    #print (confusion_matrix(y_test, y_pred))
    #target_names = ['class 0', 'class 1', 'class 2']
    #print(classification_report(y_test, y_pred, target_names=target_names))
    

In [19]:
model_making("Random Forest",vect, model3, X_train, y_train, X_test, y_test)

/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` 

In [20]:
model_making("SVM" , vect, model4, X_train, y_train, X_test, y_test)

/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:119: UserWarning: indptr array has non-integer dtype (float64)
  % self.indptr.dtype.name)


In [21]:
model_making("Logistic Regression",vect, model5, X_train, y_train, X_test, y_test)

/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:130: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.indices) != 1 or np.rank(self.indptr) != 1:
/usr/lib/python3/dist-packages/scipy/sparse/coo.py:200: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if np.rank(self.data) != 1 or np.rank(self.row) != 1 or np.rank(self.col) != 1:


In [22]:
#model_making("Naive Bayes", vect , model2 , X_train, y_train, X_test, y_test )

In [23]:
print("Total dataset",len(X))
print("Training dataset: ",len(X_train))
print("Testing dataset: ",len(X_test),"\n")
print("{:20} {:^20} {:^20} {:^20}\n ".format("Name" , "Accuracy" , "Training Time(s)" , "Prediction Time(s)" ) )

for i in range(len(name)):
    print("{:20} {:^20.3f} {:^20.3f} {:20.3f}s \n ".format(name[i] , results[i] , training_time[i] , prediction_time[i] ) )

Total dataset 19
Training dataset:  14
Testing dataset:  5 

Name                       Accuracy         Training Time(s)    Prediction Time(s) 
 
Random Forest               20.000               1.526                        0.209s 
 
SVM                         20.000               0.012                        0.004s 
 
Logistic Regression         20.000               0.021                        0.003s 
 


In [43]:
X_test


11    ['male', '22', 'product', 'type', 'term', 'ter...
13    ['1995', '23', 'product', 'type', 'term', 'fac...
16    ['female', '1996', '22', 'product', 'type', 't...
9     ['female', '1950', '35', 'product', 'type', 't...
8                     ['female', '1950', '68', '255lb']
Name: Lemmitize, dtype: object